In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, ConvLSTM2D, GRU
from keras.models import Model
from keras.models import model_from_json
from keras.initializers import Constant
from keras.utils.vis_utils import plot_model
import pickle as pkl
import pydot, pydotplus
from sklearn.externals import joblib

Using TensorFlow backend.


In [2]:
BASE_DIR = '../Glove'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.3

In [3]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [4]:
print(embeddings_index['food'])

[ 0.47222   -0.44545   -0.51833   -0.26818    0.44427   -0.25108
 -0.99282   -0.90198    1.8729     0.039081   0.14284    0.074878
  1.0543    -0.3203     1.0722     0.44323    0.0099484  0.15754
  0.51399   -0.77668    0.924      0.010958   0.58815    0.23078
 -0.34281   -0.88444   -0.31492    0.12661    1.1445     0.60775
  3.4344     0.63561   -0.13832    0.28045   -0.16181    0.77541
 -0.49888    0.4602     0.91799    0.29007    0.06884    0.59978
  0.53967   -0.061752   1.2975     0.92323   -0.80945    0.34932
  0.33934    0.25499  ]


In [5]:
infile = open("Texts.pkl", "rb")
texts = pkl.load(infile)
infile2 = open("score.pkl", "rb")
labels = pkl.load(infile2)

In [6]:
print(type(texts))
print(type(texts[0]))
print(len(texts))
print(type(labels))
print(type(labels[0]))
print(len(labels))

<class 'list'>
<class 'str'>
393931
<class 'list'>
<class 'int'>
393931


In [7]:
# finally, vectorize the text samples into a 2D integer tensor

#tokenization
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

#padding the sequences so that they must have equal length
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 74580 unique tokens.
Shape of data tensor: (393931, 100)
Shape of label tensor: (393931, 3)


In [8]:
print(type(sequences))
print(type(word_index))
print(sequences[0])

<class 'list'>
<class 'dict'>
[2671, 519, 2664, 1825, 1366, 1063, 11730, 5921, 171, 220, 816, 816, 679, 816, 171, 220, 168, 9, 1104, 3628, 845, 138, 50, 8054, 5634, 1406, 27, 1665, 2576, 638, 14, 1450, 2643, 229, 252, 441, 4, 12818, 1104, 1, 953, 70, 945, 105, 284, 77, 305, 898, 53, 2664]


In [9]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


In [10]:
# prepare embedding matrix
print('Preparing embedding matrix.')
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Preparing embedding matrix.


In [11]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


In [12]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [29]:
# train a 1D convnet with global maxpooling
print('Configuring and compiling CNN model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling CNN model..


In [14]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model...
Train on 275752 samples, validate on 118179 samples
Epoch 1/10
275752/275752 [==============================] - 662s 2ms/step - loss: 0.6269 - acc: 0.7804 - val_loss: 0.6346 - val_acc: 0.7743
Epoch 2/10
275752/275752 [==============================] - 666s 2ms/step - loss: 0.5848 - acc: 0.7886 - val_loss: 0.5857 - val_acc: 0.7890
Epoch 3/10
275752/275752 [==============================] - 672s 2ms/step - loss: 0.5646 - acc: 0.7940 - val_loss: 0.6027 - val_acc: 0.7885
Epoch 4/10
275752/275752 [==============================] - 672s 2ms/step - loss: 0.5474 - acc: 0.7992 - val_loss: 0.5944 - val_acc: 0.7893
Epoch 5/10
275752/275752 [==============================] - 662s 2ms/step - loss: 0.5314 - acc: 0.8050 - val_loss: 0.6084 - val_acc: 0.7796
Epoch 6/10
275752/275752 [==============================] - 671s 2ms/step - loss: 0.5178 - acc: 0.8097 - val_loss: 0.6067 - val_acc: 0.7788
Epoch 7/10
275752/275752 [==============================] - 674s 2ms/step - loss: 0.5057 -

In [17]:
# train a 1D convnet with global maxpooling
print('Configuring and compiling LSTM model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling LSTM model..


In [19]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 275752 samples, validate on 118179 samples
Epoch 1/5
275752/275752 [==============================] - 1090s 4ms/step - loss: 0.5632 - acc: 0.7948 - val_loss: 0.5005 - val_acc: 0.8144
Epoch 2/5
275752/275752 [==============================] - 1085s 4ms/step - loss: 0.5011 - acc: 0.8164 - val_loss: 0.4815 - val_acc: 0.8193
Epoch 3/5
275752/275752 [==============================] - 1083s 4ms/step - loss: 0.4772 - acc: 0.8253 - val_loss: 0.4609 - val_acc: 0.8359
Epoch 4/5
275752/275752 [==============================] - 1082s 4ms/step - loss: 0.4621 - acc: 0.8304 - val_loss: 0.4375 - val_acc: 0.8402
Epoch 5/5
275752/275752 [==============================] - 1082s 4ms/step - loss: 0.4524 - acc: 0.8335 - val_loss: 0.4246 - val_acc: 0.8428


In [21]:
filename = 'lstm_model2.sav'
joblib.dump(model, filename)

TypeError: can't pickle SwigPyObject objects

In [23]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
# training LSTM layer with the output of an Convnet layer
print('Configuring and compiling CNN-LSTM model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling LSTM model..


In [16]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 254920 samples, validate on 109251 samples
Epoch 1/5
254920/254920 [==============================] - 1027s 4ms/step - loss: 0.3426 - acc: 0.8600 - val_loss: 0.3238 - val_acc: 0.8568
Epoch 2/5
254920/254920 [==============================] - 1007s 4ms/step - loss: 0.2863 - acc: 0.8830 - val_loss: 0.2580 - val_acc: 0.8953
Epoch 3/5
254920/254920 [==============================] - 1008s 4ms/step - loss: 0.2635 - acc: 0.8932 - val_loss: 0.2441 - val_acc: 0.9030
Epoch 4/5
254920/254920 [==============================] - 1017s 4ms/step - loss: 0.2503 - acc: 0.8986 - val_loss: 0.2337 - val_acc: 0.9052
Epoch 5/5
254920/254920 [==============================] - 1179s 5ms/step - loss: 0.2420 - acc: 0.9027 - val_loss: 0.2259 - val_acc: 0.9090


In [17]:
# serialize weights to HDF5
model.save_weights("hybrid-model-1.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
# CNN-LSTM model-2
print('Configuring and compiling CNN-LSTM model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = MaxPooling1D(5)(x)
x = LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling CNN-LSTM model..


In [32]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [23]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 254920 samples, validate on 109251 samples
Epoch 1/5
254920/254920 [==============================] - 1049s 4ms/step - loss: 0.3420 - acc: 0.8606 - val_loss: 0.3181 - val_acc: 0.8715
Epoch 2/5
254920/254920 [==============================] - 1012s 4ms/step - loss: 0.2853 - acc: 0.8832 - val_loss: 0.2646 - val_acc: 0.8930
Epoch 3/5
254920/254920 [==============================] - 1013s 4ms/step - loss: 0.2630 - acc: 0.8930 - val_loss: 0.2410 - val_acc: 0.9018
Epoch 4/5
254920/254920 [==============================] - 1030s 4ms/step - loss: 0.2498 - acc: 0.8990 - val_loss: 0.2314 - val_acc: 0.9059
Epoch 5/5
254920/254920 [==============================] - 1029s 4ms/step - loss: 0.2403 - acc: 0.9032 - val_loss: 0.2275 - val_acc: 0.9084


In [27]:
# training GRU layer with the output of an Convnet layer
# CNN-GRU model-1
print('Configuring and compiling CNN-LSTM model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = MaxPooling1D(5)(x)
x = GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling CNN-GRU model..


In [ ]:
plot_model(model, to_file='model_plot2.png', show_shapes=True, show_layer_names=True)

In [28]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 254920 samples, validate on 109251 samples
Epoch 1/5
254920/254920 [==============================] - 1122s 4ms/step - loss: 0.3268 - acc: 0.8673 - val_loss: 0.2618 - val_acc: 0.8926
Epoch 2/5
254920/254920 [==============================] - 962s 4ms/step - loss: 0.2704 - acc: 0.8893 - val_loss: 0.2576 - val_acc: 0.8960
Epoch 3/5
254920/254920 [==============================] - 941s 4ms/step - loss: 0.2535 - acc: 0.8971 - val_loss: 0.2321 - val_acc: 0.9059
Epoch 4/5
254920/254920 [==============================] - 937s 4ms/step - loss: 0.2438 - acc: 0.9009 - val_loss: 0.2266 - val_acc: 0.9075
Epoch 5/5
254920/254920 [==============================] - 937s 4ms/step - loss: 0.2372 - acc: 0.9040 - val_loss: 0.2206 - val_acc: 0.9103


In [13]:
# CNN-LSTM model-2 for dataset with neutral reviews
print('Configuring and compiling CNN-LSTM model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(x)
x = MaxPooling1D(5)(x)
x = LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling CNN-LSTM model..


In [14]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 275752 samples, validate on 118179 samples
Epoch 1/5
275752/275752 [==============================] - 1105s 4ms/step - loss: 0.5661 - acc: 0.7950 - val_loss: 0.5037 - val_acc: 0.8187
Epoch 2/5
275752/275752 [==============================] - 1180s 4ms/step - loss: 0.5022 - acc: 0.8161 - val_loss: 0.4636 - val_acc: 0.8306
Epoch 3/5
275752/275752 [==============================] - 1256s 5ms/step - loss: 0.4769 - acc: 0.8257 - val_loss: 0.4554 - val_acc: 0.8365
Epoch 4/5
275752/275752 [==============================] - 1260s 5ms/step - loss: 0.4624 - acc: 0.8298 - val_loss: 0.4348 - val_acc: 0.8398
Epoch 5/5
275752/275752 [==============================] - 1090s 4ms/step - loss: 0.4534 - acc: 0.8333 - val_loss: 0.4303 - val_acc: 0.8415


In [15]:
# CNN-GRU model-1 for dataset with neutral reviews
print('Configuring and compiling CNN-GRU model..')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu', data_format='channels_first')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(embedded_sequences)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Configuring and compiling CNN-GRU model..


In [16]:
print("Training model...")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_val, y_val))

Training model...
Train on 275752 samples, validate on 118179 samples
Epoch 1/5
275752/275752 [==============================] - 1094s 4ms/step - loss: 0.5439 - acc: 0.8030 - val_loss: 0.4826 - val_acc: 0.8235
Epoch 2/5
275752/275752 [==============================] - 1103s 4ms/step - loss: 0.4842 - acc: 0.8226 - val_loss: 0.4900 - val_acc: 0.8213
Epoch 3/5
275752/275752 [==============================] - 999s 4ms/step - loss: 0.4657 - acc: 0.8282 - val_loss: 0.4612 - val_acc: 0.8295
Epoch 4/5
275752/275752 [==============================] - 996s 4ms/step - loss: 0.4544 - acc: 0.8327 - val_loss: 0.4301 - val_acc: 0.8404
Epoch 5/5
275752/275752 [==============================] - 994s 4ms/step - loss: 0.4472 - acc: 0.8351 - val_loss: 0.4274 - val_acc: 0.8415
